# Part-1 : Section A
Training a multi-exit ElasticBERT model on SST-2 dataset

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES']='2'
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"


In [3]:
# The code closely follows the original ElasticBERT repository
# Feature to train models with a given exit configuration is added
# !git clone https://github.com/MLiONS/MutiExitDNNs.git


%cd /home/divya/UBERT/MutiExitDNNs/ElasticBERT
!pip install -r requirements.txt

[WinError 3] The system cannot find the path specified: '/Desktop/divya/UBERT/MutiExitDNNs/ElasticBERT'
C:\Users\divya\Desktop\divya\UBERT


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#All the hyper-parameters/ location to training dataset are set in
#MultiExitDNNs -> finetune-dynamic -> finetune_elue_entropy.sh file

#1)Set the correct location to SST-2 dataset
#All models are trained on SST-2 "train" split and evaluated on "dev" split
#"train.tsv" and "dev.tsv" are expected to be in ELUE_DIR/TASK_NAME
#You can set both ELUE_DIR and TASK_NAME in finetune_elue_entropy.sh
#Or change the dataset directory using "data_dir" option

#2)Please change the "num_output_layers" option as per the desired exit-configuration

#3)Model checkpoints will be saved at "output_dir" and
#logs will be available at "log_dir"
# !bash /home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/finetune_elue_entropy.sh

# Part-1 : Section B
Generating the prediction matrix

In [4]:
#Evaluation on other datasets-IMDb or Yelp

In [5]:
from transformers import BertTokenizer as ElasticBertTokenizer

#Set the current directory location inside "finetune-dynamic" folder
%cd /home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic

from models.configuration_elasticbert import ElasticBertConfig
from models.modeling_elasticbert_entropy import ElasticBertForSequenceClassification

/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic


2023-03-27 12:48:54.070235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 12:48:54.070271: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
#Set location to the best performing model
#Model checkpoints are saved at "output_dir" from Part-1: Section A
checkpoint_snli = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/SNLI/checkpoint-25700'
checkpoint_sst = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/SST-2/checkpoint-300'
checkpoint_mrpc = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/MRPC/checkpoint-575'
checkpoint_scitail = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/SciTail/checkpoint-3690'
checkpoint_rte = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/RTE/checkpoint-390'
checkpoint_mnli = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/MNLI/checkpoint-61360'
checkpoint_qnli = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/QNLI/checkpoint-16370'
checkpoint_qqp = '/home/divya/UBERT/MutiExitDNNs/ElasticBERT/finetune-dynamic/ckpts/elue/entropy/QQP/checkpoint-56855'


In [7]:
config = ElasticBertConfig.from_pretrained(checkpoint_mnli)
tokenizer = ElasticBertTokenizer.from_pretrained(checkpoint_mnli)
model = ElasticBertForSequenceClassification.from_pretrained(checkpoint_mnli)
#model

In [8]:
def get_args(arg_vec):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument(
        "--num_hidden_layers",
        default=None,
        type=int,
        required=True,
        help='The number of layers to import.',
    )
    parser.add_argument(
        "--num_output_layers",
        nargs = 12,
        default=None,
        type=int,
        required=True,
        help='The number of layers to output.',
    )
    parser.add_argument(
        "--data_dir",
        default=None,
        type=str,
        required=True,
        help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        required=True,
        help="Path to pre-trained model or shortcut name.",
    )
    parser.add_argument(
        "--task_name",
        default=None,
        type=str,
        required=True,
        help="The name of the task to train selected in the list.",
    )
    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory where the model predictions and checkpoints will be written.",
    )
    parser.add_argument(
        "--log_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory where the logs will be written.",
    )
    parser.add_argument(
        "--spec_eval",
        default=None,
        type=str,
        required=False,
        help="'Set as train or test based on specific split on which to evaluate'"
    )
    parser.add_argument(
        "--patience",
        default='0',
        type=str,
        required=False,
    )
    parser.add_argument(
        "--regression_threshold",
        default=0,
        type=float,
        required=False,
    )
    parser.add_argument(
        "--early_exit_entropy",
        default='0.1',
        type=str,
        required=False,
    )
    # Other parameters
    parser.add_argument(
        "--load",
        default=None,
        type=str,
        help="The path of ckpts used to continue training."
    )
    parser.add_argument(
        "--config_name",
        default="",
        type=str,
        help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        default="",
        type=str,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--cache_dir",
        default="",
        type=str,
        help="Where do you want to store the pre-trained models downloaded from huggingface.co",
    )
    parser.add_argument(
        "--max_seq_length",
        default=128,
        type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer "
             "than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument("--debug", action="store_true", help="Whether to use debug mode.")
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument(
        "--evaluate_during_training",
        action="store_true",
        help="Run evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case",
        action="store_true",
        help="Set this flag if you are using an uncased model.",
    )
    parser.add_argument(
        "--per_gpu_train_batch_size",
        default=8,
        type=int,
        help="Batch size per GPU/CPU for training.",
    )
    parser.add_argument(
        "--per_gpu_eval_batch_size",
        default=1,
        type=int,
        help="Batch size per GPU/CPU for evaluation.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument(
        "--learning_rate",
        default=5e-5,
        type=float,
        help="The initial learning rate for Adam.",
    )
    parser.add_argument("--weight_decay", default=0.01, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs",
        default=3.0,
        type=float,
        help="Total number of training epochs to perform.",
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
    parser.add_argument("--warmup_rate", default=0, type=float, help="Linear warmup over warmup_rate.")

    parser.add_argument("--logging_steps", type=int, default=500, help="Log every X updates steps.")
    parser.add_argument(
        "--save_steps",
        type=int,
        default=500,
        help="Save checkpoint every X updates steps.",
    )
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir",
        action="store_true",
        help="Overwrite the content of the output directory",
    )
    parser.add_argument(
        "--overwrite_cache",
        action="store_true",
        help="Overwrite the cached training and evaluation sets",
    )
    parser.add_argument(
        "--not_save_model",
        action="store_true",
        help="Do not save model checkpoints"
    )
    parser.add_argument("--seed", type=int, default=6, help="random seed for initialization")

    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument(
        "--fp16_opt_level",
        type=str,
        default="O1",
        help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
             "See details at https://nvidia.github.io/apex/amp.html",
    )
    parser.add_argument(
        "--local_rank",
        type=int,
        default=-1,
        help="For distributed training: local_rank",
    )
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")
    args = parser.parse_args(arg_vec)

    return args

In [9]:
from load_data import (
    load_and_cache_examples_glue,
    load_and_cache_examples_elue,
)

def evaluate_elue_entropy(args, model, tokenizer, prefix="", eval_highway=False, entropy=0.):
    model.elasticbert.set_early_exit_entropy(entropy)
    model.elasticbert.set_eval_state(eval_highway)
    model.elasticbert.reset_stats()

    eval_task = args.task_name.lower()
    eval_output_dir = args.output_dir

    num_op_layers = args.num_output_layers

    results = {}
    results_all = []
    exit_layer = []
    for i in range(sum(num_op_layers)):
        results_all.append({})

    if args.spec_eval:
      eval_dataset = load_and_cache_examples_elue(args, eval_task, tokenizer, data_type=args.spec_eval)
    else:
      eval_dataset = load_and_cache_examples_elue(args, eval_task, tokenizer, data_type='train')

    if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # multi-gpu eval
    if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    preds_all = []
    pred_tuple = []
    for i in range(sum(num_op_layers)):
        preds_all.append(None)
        pred_tuple.append(None)
    out_label_ids = None

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[-1],
            }
            inputs["token_type_ids"] = batch[2]
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if out_label_ids is None:
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
        if not eval_highway:
            for i, pred in enumerate(preds_all):
                if pred is None:
                    preds_all[i] = logits[i].detach().cpu().numpy()
                else:
                    preds_all[i] = np.append(pred, logits[i].detach().cpu().numpy(), axis=0)
        else:
            if preds is None:
                preds = logits.detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args.output_mode == "classification":
        if not eval_highway:
            for i, pred in enumerate(preds_all):
                preds_all[i] = np.argmax(pred, axis = 1)
                pred_tuple[i] = pred
        else:
            preds = np.argmax(preds, axis = 1)
            pred_tuple[i] = pred

    elif args.output_mode == "regression":
        if not eval_highway:
            for i, pred in enumerate(preds_all):
                preds_all[i] = np.squeeze(pred)
        else:
            preds = np.squeeze(preds)

    if not eval_highway:
        for i, pred in enumerate(preds_all):
            if eval_task == 'rte' or 'qnli' or 'wnli' or 'qqp':
                eval_task = 'scitail'
            if eval_task == 'mnli':
                eval_task = 'snli'
            if eval_task == 'yelp':
              eval_task = 'imdb'
            result = elue_compute_metrics(eval_task, pred, out_label_ids)
            results_all[i].update(result)

    else:
        if eval_task == 'rte' or 'qnli' or 'wnli' or 'qqp':
                eval_task = 'scitail'
        if eval_task == 'mnli':
                eval_task = 'snli'
        result = elue_compute_metrics(eval_task, preds, out_label_ids)
        results.update(result)

        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            print("  %s = %s" % (key, str(result[key])))

        exiting_layer_every_ins = model.elasticbert.exiting_layer_every_ins
        exit_layer.append(exiting_layer_every_ins)

    if eval_highway:
        speed_up = model.elasticbert.log_stats()
        return results, speed_up, exit_layer

    if args.spec_eval:
      return results_all, preds_all, pred_tuple, out_label_ids

    return results_all, preds_all, pred_tuple , out_label_ids
    #return results_all, preds_all, out_label_ids


In [10]:
ELUE_DIR='/home/divya/UBERT/elue_data'
TASK_NAME='MNLI'

arg_vec= ['--model_name_or_path', 'fnlp/elasticbert-base',
  '--task_name', 'MNLI', \
  '--do_train', \
  '--do_lower_case', \
  '--data_dir', "/home/divya/UBERT/elue_data", \
  '--log_dir', '/home/divya/UBERT/ElasticBERT/logs/elue/entropy/SNLI-BTestCheck', \
  '--output_dir', '/home/divya/UBERT/ElasticBERT/ckpts/elue/entropy/SNLI-BTestCheck', \
  '--num_hidden_layers', '12', \
  '--num_output_layers', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '1', \
  '--max_seq_length', '128', \
  '--per_gpu_train_batch_size', '32', \
  '--per_gpu_eval_batch_size',' 32', \
  '--learning_rate', '2e-5', \
  '--weight_decay', '0.1', \
  '--save_steps', '50', \
  '--logging_steps', '50', \
  '--num_train_epochs', '5',  \
  '--warmup_rate', '0.06', \
  '--evaluate_during_training', \
  '--overwrite_output_dir'
]

import argparse
parser = argparse.ArgumentParser()

args = get_args(arg_vec)

In [11]:
import torch
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    args.n_gpu = 1
args.device = device

args.output_mode = 'classification'

print(args.device)
model.to(args.device)

cuda


ElasticBertForSequenceClassification(
  (elasticbert): ElasticBertModel(
    (embeddings): ElasticBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-08, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElasticBertEncoder(
      (layer): ModuleList(
        (0): ElasticBertLayer(
          (attention): ElasticBertAttention(
            (self): ElasticBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElasticBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)


In [12]:
# !git clone https://github.com/hsm207/imdb_data.git

In [13]:
# %cd imdb_data
# !pip install -r requirements.txt

In [14]:
# !tf_upgrade_v2 --infile create_imdb_dataset.py --outfile bar.py

In [15]:
# !python bar.py --output_dir /home/divya/UBERT/elue_data/imdb_data/imdb_dataset

In [16]:
#Custom Selection
dataset = 'SNLI'#'IMDb' #or 'Yelp'

#To check model performance on SST-2 dev split:
#Please set dataset = 'SST-2' and data_split='dev'

In [17]:
import numpy as np
import pandas as pd

def get_preds(eval_dataset='SNLI', data_split='train'):
  args.spec_eval = False
  args.task_name=eval_dataset.lower()
  args.data_dir=ELUE_DIR + '/'+args.task_name

  results_all, exit_preds, pred_tuple, op_labels = evaluate_elue_entropy(args, model, tokenizer)


  # exit_preds_list = np.stack(exit_preds, axis=1)
  # df = pd.DataFrame((exit_preds_list) )
  # df['op_labels'] = op_labels

  return  results_all, exit_preds, pred_tuple, op_labels

In [18]:
from elue import elue_compute_metrics
import logging
logger = logging.getLogger(__name__)
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
import os
from tqdm import tqdm
results, final_preds, pred_tuple, op_labels = get_preds(eval_dataset=dataset, data_split='train')

Evaluating: 100%|██████████| 17168/17168 [26:47<00:00, 10.68it/s]


In [19]:
(len(pred_tuple[-1]))

549367

In [20]:
# accurac_imd = []
# for j in range(12):
#     accuracy = 0
#     for i in range(len(op_labels)):
#         if final_preds[j][i] == op_labels[i]:
#             accuracy+=1
#         else:
#             pass
#     accurac_imd.append(accuracy/len(op_labels))

In [21]:
# accurac_yelp = []
# for j in range(12):
#     accuracy = 0
#     for i in range(len(op_labels)):
#         if final_preds[j][i] == op_labels[i]:
#             accuracy+=1
#         else:
#             pass
#     accurac_yelp.append(accuracy/len(op_labels))

In [22]:
# import matplotlib.pyplot as plt
# x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# plt.plot(x, accurac_imd, linestyle='dotted', marker = '^', color="r")
# plt.plot(x, accurac_yelp, linestyle='dotted', marker = 'o', color="g")
# plt.xlabel("Layer number", fontsize = 17)
# plt.ylabel("Accuracy", fontsize = 17)
# plt.title("Accuracy vs Layer", fontsize = 17)
# plt.legend(['IMDb', 'Yelp'],
#         prop = {'size' : 15},
#         loc = 'lower right', shadow = True,
#         facecolor = 'white')
# plt.xticks(fontsize=13)
# plt.yticks(fontsize=13)
# plt.savefig("/home/divya/UBERT/Result_plot/Accuracy_vs_layer_Imdb and yelp.pdf")
# plt.show()

In [23]:
import numpy as np
import math
import scipy
def softmax(x):
    return(np.exp(x)/np.exp(x).sum())

In [24]:
pred_prob_thi = []
for i in range(len(pred_tuple[0])):
    pred_prob_thi.append(max(softmax(pred_tuple[2][i])))
# pred_proba_thi

In [25]:
pred_prob_six = []
for i in range(len(pred_tuple[0])):
    pred_prob_six.append(max(softmax(pred_tuple[5][i])))
# pred_proba_fou

In [26]:
pred_prob_las = []
for i in range(len(pred_tuple[0])):
    pred_prob_las.append(max(softmax(pred_tuple[-1][i])))
pred_prob_las

[0.99910575,
 0.9994923,
 0.9529833,
 0.96982104,
 0.98246974,
 0.99934644,
 0.90339124,
 0.98659974,
 0.9907664,
 0.7798163,
 0.9981209,
 0.9753172,
 0.997316,
 0.99983996,
 0.98320127,
 0.9926979,
 0.9994368,
 0.73188215,
 0.9937423,
 0.9989383,
 0.98298115,
 0.994629,
 0.9995364,
 0.98732424,
 0.9991029,
 0.6196092,
 0.81272244,
 0.997546,
 0.99068004,
 0.9963358,
 0.81837183,
 0.9871621,
 0.9980643,
 0.5162542,
 0.96856296,
 0.9967236,
 0.98666567,
 0.99731016,
 0.99883467,
 0.9669747,
 0.92535925,
 0.99955,
 0.8710396,
 0.9536737,
 0.9972004,
 0.9927417,
 0.9981533,
 0.9980363,
 0.99644864,
 0.47906926,
 0.9211222,
 0.99983925,
 0.9974528,
 0.97958136,
 0.9222829,
 0.6592883,
 0.9963254,
 0.99809754,
 0.9966623,
 0.97587,
 0.95984626,
 0.9932312,
 0.9685359,
 0.9912516,
 0.9539883,
 0.8702022,
 0.9243329,
 0.99241656,
 0.9993096,
 0.9988882,
 0.95071775,
 0.6482637,
 0.9972621,
 0.6541429,
 0.8965362,
 0.78879297,
 0.99505806,
 0.998071,
 0.9947783,
 0.9926926,
 0.99054146,
 0.932

In [27]:
len(final_preds[0])

549367

In [28]:
max(pred_prob_thi)

0.9996928

In [47]:
op_labels_1 = []
for i in op_labels:
    if i == 0:
        op_labels_1.append(1)
    elif i==1:
        op_labels_1.append(2)
    else:
        op_labels_1.append(0)

In [48]:
df = pd.DataFrame(list(zip(final_preds[2],final_preds[5], final_preds[-1], pred_prob_thi, pred_prob_six, pred_prob_las, op_labels_1)), columns =['Thi_layer_P','Six_layer_P', 'Last_layer','PProb_thi', 'PProb_six', 'PProb_las', 'True_labels'])

In [49]:
print(df.shape[0])

549367


In [50]:
# df1 = pd.DataFrame(list(zip(final_preds[0], final_preds[1], final_preds[2], final_preds[3], final_preds[4], final_preds[5], final_preds[6], final_preds[7], final_preds[8], final_preds[9], final_preds[10], final_preds[11], op_labels)), columns =['Fir_p', 'Sec_p', 'Thi_p', 'Fou_p', 'Fiv_p', 'Six_p', 'Sev_p', 'Eig_p', 'Nin_p', 'Ten_p', 'Ele_p', 'Twe_p', 'True_labels'])

In [51]:
# df2 = pd.DataFrame(list(zip(final_preds[0], final_preds[1], final_preds[2], final_preds[3], final_preds[4], final_preds[5], final_preds[6], op_labels)), columns =['Fir_p', 'Thi_p', 'Fou_p', 'Fiv_p', 'Sev_p', 'Nin_p', 'Twe_p', 'True_labels'])

In [52]:
(df.head(5))

,Thi_layer_P,Six_layer_P,Last_layer,PProb_thi,PProb_six,PProb_las,True_labels
0,2,2,2,0.983691,0.998154,0.999106,2
1,0,0,0,0.973306,0.999317,0.999492,0
2,2,1,1,0.409512,0.951424,0.952983,1
3,2,2,2,0.891311,0.967909,0.969821,2
4,1,1,1,0.828995,0.971407,0.982470,1


In [53]:
accuracy_imdb = []
for j in df.columns:
  accuracy = 0
  for i in range(df.shape[0]):
      if df[j][i] == df['True_labels'][i]:
          accuracy += 1
      else:
          pass
  print("Accuracy = ", accuracy/df.shape[0])
  accuracy_imdb.append(accuracy)

Accuracy =  0.7347419848662188
Accuracy =  0.7943032617539824
Accuracy =  0.7971337921644365
Accuracy =  0.0
Accuracy =  0.0
Accuracy =  0.0
Accuracy =  1.0


In [54]:
# accuracy_yelp = []
# for j in df2.columns:
#   accuracy = 0
#   for i in range(df1.shape[0]):
#       if df2[j][i] == df2['True_labels'][i]:
#           accuracy += 1
#       else:
#           pass
#   print("Accuracy = ", accuracy/df2.shape[0])
#   accuracy_imdb.append(accuracy)

In [55]:
df.to_csv("/home/divya/UBERT/CSV_files/Early_Exit_Confidence_data_snli_max_exits(3,6,12)_difference.csv")

In [137]:
# df.to_csv(r"/UBERT/CSV_files/Early_Exit_Confidence_data_SST2_new_exits(12,12)_difference.csv")

In [ ]:
df_train = get_preds(eval_dataset=dataset, data_split='train')
df_test = get_preds(eval_dataset=dataset, data_split='test')

df_tot = pd.concat([df_train, df_test])
df_tot = df_tot.reset_index(drop=True)
print(df_tot.head())

df_tot.to_csv(r'/content/drive/MyDrive/Early_Exits_Divya/Model_exit_predictions/Exit_Predictions_TrainTest_IMDb_8exits.csv',sep ='\t', index = False)

Evaluating: 100%|██████████| 782/782 [03:10<00:00,  4.10it/s]

   0  1  2  3  4  5  6  7  op_labels
0  0  0  0  1  0  0  0  1          1
1  0  0  0  1  0  0  0  0          1
2  0  0  0  1  0  0  0  0          1
3  0  0  0  1  0  0  1  1          1
4  0  0  1  1  0  0  1  1          1
